In [46]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

In [47]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()

# giảm chỉ số đi 1 ở 2 cột id đầu
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [ ]:
class CF(object):
    def __init__(self, Y_data, k, dist_func=cosine_similarity, uuCF=1):
        self.uuCF = uuCF # 1: User-User CF, 0: Item-Item CF
        
        # Giữ nguyên nếu là User-User, đảo cột (user <-> item) nếu là Item-Item
        self.Y_data = Y_data if uuCF else Y_data[:, [1, 0, 2]]
        
        self.k = k # Số lượng điểm lân cận (hàng xóm) để xét
        self.dist_func = dist_func # Hàm tính độ tương đồng
        self.Ybar_data = None # Dữ liệu sau khi chuẩn hóa (sẽ tính sau)
        
        # Tính tổng số user và item (phải cộng 1 vì index bắt đầu từ 0)
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1 
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1

    def add(self, new_data):
        # Thêm dòng data mới vào cuối ma trận
        self.Y_data = np.concatenate((self.Y_data, new_data), axis = 0)
    
    def normalize_Y(self):
        users = self.Y_data[:, 0] # Lấy toàn bộ cột user_id
        self.Ybar_data = self.Y_data.copy() # Bản sao để chứa dữ liệu chuẩn hóa
        self.mu = np.zeros((self.n_users,)) # Mảng lưu điểm trung bình của từng user
        
        for n in range(self.n_users):
            # Tìm các dòng dữ liệu (index) thuộc về user n
            ids = np.where(users == n)[0].astype(np.int32)
            # Lấy điểm rating thực tế của user n
            ratings = self.Y_data[ids, 2]
            # Tính điểm trung bình của user n
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # Xử lý lỗi nếu user chưa rate phim nào (NaN)
            self.mu[n] = m # Gán điểm trung bình m vào mảng mu
            # Chuẩn hóa: Điểm thực tế trừ đi điểm trung bình
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

        # Tạo ma trận thưa (Sparse matrix) kích thước (n_items, n_users)
        # Chỉ lưu các giá trị khác 0 để tiết kiệm RAM
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
            (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
        self.Ybar = self.Ybar.tocsr() # Chuyển sang định dạng CSR để tính toán nhanh hơn
    
    def similarity(self):
        # Tính ma trận độ tương đồng (Cosine Similarity) 
        # Ybar.T là ma trận chuyển vị (chuyển (item, user) thành (user, item))
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    
    def refresh(self):
            # Thực hiện lại 2 hàm phía trên khi có thêm dữ liệu
            normalize_Y(self)
            similarity(self) 
    
    def __pred(self, u, i, normalized = 1):
            # Bước 1: Tìm tất cả các dòng dữ liệu mà item i được đánh giá
            ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
            
            # Bước 2: Lấy ra danh sách các user đã đánh giá item i đó
            users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
            
            # Bước 3: Lấy độ tương đồng (similarity) giữa user u và những user ở Bước 2
            sim = self.S[u, users_rated_i]
            
            # Bước 4: Tìm k user có độ tương đồng cao nhất (hàng xóm gần nhất)
            # np.argsort sắp xếp tăng dần, [-self.k:] lấy k giá trị cuối cùng (lớn nhất)
            a = np.argsort(sim)[-self.k:] 
            
            # Lấy ra các giá trị độ tương đồng tương ứng của k user này
            nearest_s = sim[a]
            
            # Lấy điểm rating (đã chuẩn hóa) của k user này cho item i
            r = self.Ybar[i, users_rated_i[a]]
            
            # Tính toán kết quả (Trung bình có trọng số)
            # Cộng thêm 1e-8 để tránh lỗi chia cho 0 (Divide by zero)
            if normalized:
                # Trả về điểm dự đoán ở dạng đã chuẩn hóa
                return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)

            # Trả về điểm dự đoán thực tế (Cộng trả lại điểm trung bình mu của user u)
            return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8) + self.mu[u]
    def pred(self, u, i, normalized = 1):
        
        # SỬA LỖI: Đổi 'normalize' thành 'normalized' cho khớp với tham số đầu vào
        if self.uuCF: 
            return self.__pred(u, i, normalized) # Nếu là User-User CF
            
        # Nếu là Item-Item CF, chỉ cần đảo ngược vị trí i và u
        return self.__pred(i, u, normalized)
    def recommend(self):
        print('Kết quả gợi ý (Recommendation): ')
        # Duyệt qua toàn bộ users (hoặc items nếu đã tráo cột)
        for u in range(self.n_users):
            # 1. Tìm index các dòng dữ liệu của u
            ids = np.where(self.Y_data[:, 0] == u)[0]
            # 2. Lấy danh sách các item mà u ĐÃ đánh giá
            items_rated_by_u = self.Y_data[ids, 1].tolist()              
            recommended_items = []
            # 3. Duyệt qua toàn bộ các item trong hệ thống
            for i in range(self.n_items):
                # Nếu item i CHƯA được u đánh giá thì mới dự đoán
                if i not in items_rated_by_u:
                    rating = self.__pred(u, i) # Dự đoán điểm rating
                    # Nếu điểm dự đoán > 0 (chuẩn hóa) thì đưa vào danh sách gợi ý
                    if rating > 0: 
                        recommended_items.append(i)
            # 4. In kết quả ngay sau khi tính xong cho từng u
            if self.uuCF:
                print(f'    Gợi ý item(s): {recommended_items} cho user {u}')
            else: 
                    print(f'    Gợi ý item {u} cho user(s): {recommended_items}')

In [49]:
def RMSE(rs, rate_test):
    n_tests = rate_test.shape[0]
    SE = 0 # Biến lưu tổng bình phương sai số (Squared Error)
    
    for n in range(n_tests):
        # Lấy u và i từ tập test để dự đoán (normalized = 0 để lấy điểm gốc)
        pred = rs.pred(rate_test[n, 0], rate_test[n, 1], normalized=0)
        
        # Điểm thực tế nằm ở cột 2 của tập test
        actual_rating = rate_test[n, 2]
        
        # Tính bình phương hiệu số giữa Dự đoán và Thực tế
        SE += (pred - actual_rating)**2 
        
    # Tính RMSE bằng cách lấy Căn bậc 2 của (Tổng sai số / Số lượng test)
    RMSE = np.sqrt(SE / n_tests)
    
    return RMSE

In [52]:
rs_uu = CF(rate_train, k=10, uuCF=1)
rs_uu.refresh() 
rmse_uu = RMSE(rs_uu, rate_test)
print(f'Kết quả User-user CF, RMSE = {rmse_uu}')

Kết quả User-user CF, RMSE = 1.00539567261509


In [53]:
rs_uu = CF(rate_train, k=10, uuCF=0)
rs_uu.refresh() 
rmse_uu = RMSE(rs_uu, rate_test)
print(f'Kết quả User-user CF, RMSE = {rmse_uu}')

c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Kết quả User-user CF, RMSE = 0.9865673282667308


Công thức Item-item tối ưu hơn trong tính toán và cho ra sai số ít hơn so với user-user